In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Reasoning Engine in Vertex AI
@forusone (shins777@gmail.com)


This lab simplified the original colab [intro_reasoning_engine](https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/intro_reasoning_engine.ipynb) |Author(s) | [Kristopher Overholt](https://github.com/koverholt) |

### Reasoning Engine in Vertex AI

[Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview) (LangChain on Vertex AI) is a managed service that helps you to build and deploy an agent reasoning framework. It gives you the flexibility to choose how much reasoning you want to delegate to the LLM and how much you want to handle with customized code. You can define Python functions that get used as tools via [Gemini Function Calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling). Reasoning Engine integrates closely with the Python SDK for the Gemini model in Vertex AI, and it can manage prompts, agents, and examples in a modular way. Reasoning Engine is compatible with LangChain, LlamaIndex, or other Python frameworks.


### Install Vertex AI SDK for Python

In [5]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

In [6]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### Initial set up

In [13]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

### Tool define

In [9]:
# Tool define
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [10]:
# Tool test
get_exchange_rate(currency_from="USD", currency_to="KRW")

{'amount': 1.0, 'base': 'USD', 'date': '2024-11-15', 'rates': {'KRW': 1394.91}}

### Agent - Local unit test

In [11]:
# Agent Define
agent = reasoning_engines.LangchainAgent(
    model=MODEL,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": False},
)

#### Test your agent locally


In [12]:
# Agent local unit test before deploying.
agent.query(input="What's the exchange rate from US dollars to Korean currency today?")

{'input': "What's the exchange rate from US dollars to Korean currency today?",
 'output': 'The exchange rate from US dollars to Korean won today is 1 USD to 1394.91 KRW.'}

### Deploy your agent on Vertex AI

In [17]:
# Crate Methods
# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_create

remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    display_name="simple agent",
    reasoning_engine_name="ai-agent", # does not support user-defined resource IDs at the moment
    gcs_dir_name = "ai-agent-1",
    description="This is a simple reasoning engine.",
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic==2.7.4",
        "requests",
    ],
    extra_packages = []
)

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket reasoning_7424
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://reasoning_7424/ai-agent-1/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://reasoning_7424/ai-agent-1/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://reasoning_7424/ai-agent-1/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/6325534375060439040/operations/1550127123887292416
INFO:vertexai.reasoning_engines._reasoning_engines:ReasoningEngine created. Resource name: projects/721521243942/locations/us-central1/reasoningEngines/6325534375060439040
INFO:vertexai.reasoning_engines._reasonin

In [21]:
# query
remote_agent.query(
    input="What's the exchange rate from US dollars to Korea won  currency today?"
)

{'input': "What's the exchange rate from US dollars to China Yuan currency today?",
 'output': 'The exchange rate from US dollars to China Yuan currency today is 1 USD to 7.2278 CNY. '}

In [33]:
# Properties of ReasoningEngine class.
# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine

#----------------------------------------
def get_reasoning_engine(display_name:str):
  try:
    for re in reasoning_engines.ReasoningEngine.list():
      if re.display_name == display_name:
        return reasoning_engines.ReasoningEngine(re.name)
      else:
        print("No such reasoning engine or Invalid display name.")

  except Exception as e:
    print(e)

#----------------------------------------
def get_re_list():
  """
  List reasoning engines.
  """

  try:
    if not reasoning_engines.ReasoningEngine.list():
      print("No reasoning engines")

    for idx, re in enumerate(reasoning_engines.ReasoningEngine.list()):
        print(f"Agent {idx}: \n\tDisplay Name [{re.display_name}] \n\tName [{re.name}] \n\tCreation Time [{re.create_time}] \n\tResource Name [{re.resource_name}]\n")

  except Exception as e:
    print(e)

#----------------------------------------

def del_re(name):
  """
  Delete a reasoning engines.
  @param name: The name of the reasoning engine.
  @type name: str
  """

  try:
    re = reasoning_engines.ReasoningEngine(name)
    re.delete()
    print(f"Deleted {name}")
  except Exception as e:
    print(e)


In [34]:
get_re_list()

Agent 0: 
	Display Name [simple agent] 
	Name [6325534375060439040] 
	Creation Time [2024-11-18 13:34:12.086807+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/6325534375060439040]



In [29]:
re = get_reasoning_engine('simple agent')
re.query(
    input="What's the exchange rate from US dollars to Korea won currency today?"
)

{'input': "What's the exchange rate from US dollars to China Yuan currency today?",
 'output': 'The exchange rate from US dollars to China Yuan today is 1 USD to 7.2278 CNY. '}